<a href="https://colab.research.google.com/github/OlivaVe/project4_team7/blob/main/Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [991 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [938 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,022 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,252 kB]
Get:13 http://archive.u

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Start using the CSV files.

In [ ]:
df = pd.read_csv('/content/sample_data/application_record.csv')

In [ ]:
df_cr = pd.read_csv('/content/sample_data/credit_record.csv')

In [ ]:
df_cr.head(3)

,ID,STATUS
0,5001711,1
1,5001712,1
2,5001713,1


In [ ]:
df.head(3) # it has 438557

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,NaN,2
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,NaN,2
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,Security staff,2


In [ ]:
df =df.drop(columns='OCCUPATION_TYPE')

# Creating only one dataframe.

In [ ]:
df_t = df.merge(df_cr, how='inner', on='ID')

In [ ]:
df_t =df_t.drop(columns='ID')

In [ ]:
df_t.head(3)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,STATUS
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,1
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,1
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2,1


In [ ]:
df_t['CODE_GENDER'].count()

24672

In [ ]:
#Changing in 'FLAG_OWN_CAR' to 1 and 0 for more easy use.
for index, row in df_t.iterrows():
    if row['FLAG_OWN_CAR'] == 'N':
        df_t.at[index, 'FLAG_OWN_CAR'] = 0
    elif row['FLAG_OWN_CAR'] == 'Y':
        df_t.at[index, 'FLAG_OWN_CAR'] = 1
#Changing in 'FLAG_OWN_REALTY' to 1 and 0 for more easy use.
for index, row in df_t.iterrows():
    if row['FLAG_OWN_REALTY'] == 'N':
        df_t.at[index, 'FLAG_OWN_REALTY'] = 0
    elif row['FLAG_OWN_REALTY'] == 'Y':
        df_t.at[index, 'FLAG_OWN_REALTY'] = 1

In [ ]:
#Visualizing the changes
[df_t['FLAG_OWN_CAR'].unique(), df_t['FLAG_OWN_REALTY'].unique()]

[array([1, 0], dtype=object), array([1, 0], dtype=object)]

In [ ]:
#Visualizing variables
df_t['CNT_CHILDREN'].unique()

array([ 0,  1,  2,  3,  4,  5, 14, 19])

In [ ]:
#Counting the number of variables
for x in range(0, 20):
  print(x,'-',(df_t[df_t['CNT_CHILDREN']==x]['DAYS_BIRTH']/365).round().count())

0 - 17323
1 - 4852
2 - 2150
3 - 295
4 - 38
5 - 10
6 - 0
7 - 0
8 - 0
9 - 0
10 - 0
11 - 0
12 - 0
13 - 0
14 - 3
15 - 0
16 - 0
17 - 0
18 - 0
19 - 1


In [ ]:
#Cleaning the data
for x in range(6, 20):
  df_t= df_t[df_t['CNT_CHILDREN'] != x]
#Showing the new result of variables
df_t['CNT_CHILDREN'].unique()

array([0, 1, 2, 3, 4, 5])

In [ ]:
df_t.head(3)# it has now 24668

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,STATUS
0,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,1
1,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2,1
2,M,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2,1


In [ ]:
# df.shape[1]
for x in (5, 6, 7,8,11,12):
  print(df_t.columns[x], '-', df_t.iloc[:, x].unique(),'\n')

NAME_INCOME_TYPE - ['Working' 'Commercial associate' 'Pensioner' 'State servant' 'Student'] 

NAME_EDUCATION_TYPE - ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree'] 

NAME_FAMILY_STATUS - ['Civil marriage' 'Married' 'Single / not married' 'Separated' 'Widow'] 

NAME_HOUSING_TYPE - ['Rented apartment' 'House / apartment' 'Municipal apartment'
 'With parents' 'Co-op apartment' 'Office apartment'] 

CNT_FAM_MEMBERS - [2 1 3 4 5 6 7] 

STATUS - [1 0] 



In [ ]:
for x in range(1, 8):
  print(x,'-',(df_t[df_t['CNT_FAM_MEMBERS']==x]['DAYS_BIRTH']/365).round().count())

1 - 4825
2 - 13279
3 - 4196
4 - 2041
5 - 281
6 - 37
7 - 9


In [ ]:
for index, row in df_t.iterrows():
  df_t.at[index, 'DAYS_BIRTH'] = ((df_t.at[index, 'DAYS_BIRTH']*-1)/365.25).round()

In [ ]:
for index, row in df_t.iterrows():
  df_t.at[index, 'DAYS_EMPLOYED'] = ((df_t.at[index, 'DAYS_EMPLOYED']*-1)/365.25).round()

In [ ]:
[df_t['DAYS_BIRTH'].unique(), df_t['DAYS_EMPLOYED'].unique()]

[array([33, 59, 52, 62, 46, 49, 29, 27, 30, 32, 56, 43, 44, 34, 55, 47, 42,
        37, 28, 61, 57, 45, 54, 40, 51, 39, 48, 24, 21, 35, 36, 26, 50, 41,
        58, 60, 63, 31, 38, 53, 22, 64, 23, 25, 65, 67, 66, 69, 68]),
 array([   12,     3,     8, -1000,     2,     4,     6,     9,    19,
           15,     5,    13,     7,    17,    30,    16,    11,     1,
           10,    24,    14,    25,     0,    20,    27,    18,    21,
           23,    22,    28,    39,    32,    33,    37,    26,    38,
           31,    29,    35,    34,    41,    40,    36,    42,    43])]

In [ ]:
for index, row in df_t.iterrows():
  if row['DAYS_EMPLOYED'] == -1000:
    df_t.at[index, 'DAYS_EMPLOYED'] = 45

df_t['DAYS_EMPLOYED'].value_counts()

45    4133
1     2226
2     2028
3     1968
4     1853
5     1609
6     1389
7     1333
8     1129
9     1066
10     810
11     621
0      619
15     503
12     495
13     451
14     333
20     235
16     220
17     201
18     176
19     171
21     157
22     136
23     110
24      94
26      84
27      81
25      80
28      71
29      44
30      41
33      33
32      29
31      27
35      24
41      24
34      21
36      13
40      12
38       7
39       4
37       3
43       3
42       1
Name: DAYS_EMPLOYED, dtype: int64

In [ ]:
df_t.rename(
    columns={"CODE_GENDER": "GENDER", "FLAG_OWN_REALTY": "FLAG_OWN_PROPERTY", "NAME_INCOME_TYPE": "INCOME_TYPE", "NAME_EDUCATION_TYPE": "EDUCATION_TYPE", "NAME_FAMILY_STATUS": "FAMILY_STATUS", "NAME_HOUSING_TYPE": "HOUSING_TYPE", "DAYS_BIRTH": "AGE", "DAYS_EMPLOYED": "YEARS_EMPLOYED", "DAYS_EMPLOYED": "YEARS_EMPLOYED", "STATUS": "APPROVAL_STATUS"},
    inplace=True,
)

In [ ]:
df_t.head(1)

,GENDER,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,INCOME_TYPE,EDUCATION_TYPE,FAMILY_STATUS,HOUSING_TYPE,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS
0,M,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,33,12,2,1


In [ ]:
df_t.to_csv('/content/sample_data/application_record_revised_tableau.csv')

# saving the CSV for Tableau

In [ ]:
columns_to_convert = [
    'GENDER', 'INCOME_TYPE', 'EDUCATION_TYPE',
    'FAMILY_STATUS', 'HOUSING_TYPE']
dummies = pd.get_dummies(df_t, columns=columns_to_convert)

In [ ]:
# dummies.columns
dummies

,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS,GENDER_F,GENDER_M,...,FAMILY_STATUS_Married,FAMILY_STATUS_Separated,FAMILY_STATUS_Single / not married,FAMILY_STATUS_Widow,HOUSING_TYPE_Co-op apartment,HOUSING_TYPE_House / apartment,HOUSING_TYPE_Municipal apartment,HOUSING_TYPE_Office apartment,HOUSING_TYPE_Rented apartment,HOUSING_TYPE_With parents
0,1,1,0,427500.0,33,12,2,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,427500.0,33,12,2,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,1,0,112500.0,59,3,2,1,0,1,...,1,0,0,0,0,1,0,0,0,0
3,0,1,0,270000.0,52,8,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0
4,0,1,0,270000.0,52,8,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24667,1,1,0,315000.0,47,7,2,0,0,1,...,1,0,0,0,0,1,0,0,0,0
24668,0,1,0,157500.0,34,4,2,1,1,0,...,1,0,0,0,0,1,0,0,0,0
24669,0,1,0,157500.0,34,4,2,1,1,0,...,1,0,0,0,0,1,0,0,0,0
24670,0,1,0,283500.0,49,2,2,0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Split our preprocessed data into our features and target arrays
X = dummies.drop(columns='APPROVAL_STATUS')
y = dummies['APPROVAL_STATUS']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Trainig the model

In [ ]:
import tensorflow as tf

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=30, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                2480      
                                                                 
 dense_7 (Dense)             (None, 80)                6480      
                                                                 
 dense_8 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9,041
Trainable params: 9,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 2s 2ms/step - loss: 0.0822 - accuracy: 0.9843
Epoch 2/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0684 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0669 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0654 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0654 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0643 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0638 - accuracy: 0.9875
Epoch 8/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0626 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0623 - accuracy: 0.9875
Epoch 10/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0619 - accuracy: 0.9876

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 1s - loss: 0.0830 - accuracy: 0.9849 - 853ms/epoch - 4ms/step
Loss: 0.08304847031831741, Accuracy: 0.9849197268486023


In [ ]:
df_t.to_csv('/content/sample_data/application_record_revised.csv')

---
Defining important features
---

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
X = dummies.drop(columns='APPROVAL_STATUS')
y = dummies['APPROVAL_STATUS']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Model -  RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(X_train_scaled, y_train)
print(forest.score(X_test_scaled, y_test))

pd.DataFrame({
    'feature': X.columns,
    'importance': forest.feature_importances_
}).sort_values(by='importance', ascending=False)

0.9858926544511107


,feature,importance
4,AGE,0.257125
3,AMT_INCOME_TOTAL,0.247788
5,YEARS_EMPLOYED,0.156321
1,FLAG_OWN_PROPERTY,0.039650
6,CNT_FAM_MEMBERS,0.037369
0,FLAG_OWN_CAR,0.034867
2,CNT_CHILDREN,0.030825
13,INCOME_TYPE_Working,0.021437
9,INCOME_TYPE_Commercial associate,0.018121
18,EDUCATION_TYPE_Secondary / secondary special,0.016598


----
# Creating a second mode
-----

In [ ]:
df_p= df_t.drop(columns = [
    'GENDER', 'INCOME_TYPE', 'EDUCATION_TYPE',
    'FAMILY_STATUS', 'HOUSING_TYPE'])
df_p

,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS
0,1,1,0,427500.0,33,12,2,1
1,1,1,0,427500.0,33,12,2,1
2,1,1,0,112500.0,59,3,2,1
3,0,1,0,270000.0,52,8,1,1
4,0,1,0,270000.0,52,8,1,1
...,...,...,...,...,...,...,...,...
24667,1,1,0,315000.0,47,7,2,0
24668,0,1,0,157500.0,34,4,2,1
24669,0,1,0,157500.0,34,4,2,1
24670,0,1,0,283500.0,49,2,2,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X = df_p.drop(columns='APPROVAL_STATUS')
y = df_p['APPROVAL_STATUS']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()
# First hidden layer
nn2.add(tf.keras.layers.Dense(units=80, input_dim=7, activation='relu'))
# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=80, activation='relu'))
# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 80)                640       
                                                                 
 dense_19 (Dense)            (None, 80)                6480      
                                                                 
 dense_20 (Dense)            (None, 1)                 81        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn2.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model
nn2.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 3s 3ms/step - loss: 0.0857 - accuracy: 0.9870
Epoch 2/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0709 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0693 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0686 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0688 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0681 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 2s 4ms/step - loss: 0.0687 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0680 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 3s 5ms/step - loss: 0.0675 - accuracy: 0.9874
Epoch 10/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0677 - accuracy: 0.9874

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 0s - loss: 0.0784 - accuracy: 0.9851 - 412ms/epoch - 2ms/step
Loss: 0.07843910902738571, Accuracy: 0.9850819110870361


----
# 3rd model
---

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()
# First hidden layer
nn3.add(tf.keras.layers.Dense(units=100, input_dim=7, activation='tanh'))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=100, activation='tanh'))
nn3.add(tf.keras.layers.Dense(units=100, activation='tanh'))
# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               800       
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 100)               10100     
                                                                 
 dense_24 (Dense)            (None, 1)                 101       
                                                                 
Total params: 21,101
Trainable params: 21,101
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn3.compile(
    loss = 'binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy']
)

In [ ]:
# Train the model
nn3.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 3s 3ms/step - loss: 0.0658 - accuracy: 0.9874
Epoch 2/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0652 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0651 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0649 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0648 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0648 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0647 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 2s 4ms/step - loss: 0.0646 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0646 - accuracy: 0.9874
Epoch 10/10
579/579 [==============================] - 1s 3ms/step - loss: 0.0645 - accuracy: 0.9874

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 0s - loss: 0.0780 - accuracy: 0.9851 - 402ms/epoch - 2ms/step
Loss: 0.07802628725767136, Accuracy: 0.9850819110870361


# Using Spark

In [ ]:
df_revised = spark.read.csv(SparkFiles.get("/content/sample_data/application_record_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

In [ ]:
df_revised.show()

+---+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+---+--------------+---------------+---------------+
|_c0|     ID|GENDER|FLAG_OWN_CAR|FLAG_OWN_PROPERTY|CNT_CHILDREN|AMT_INCOME_TOTAL|         INCOME_TYPE|      EDUCATION_TYPE|       FAMILY_STATUS|     HOUSING_TYPE|AGE|YEARS_EMPLOYED|CNT_FAM_MEMBERS|APPROVAL_STATUS|
+---+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+---+--------------+---------------+---------------+
|  0|5008804|     M|           1|                1|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment| 33|            12|              2|              1|
|  1|5008805|     M|           1|                1|           0|        427500.0|             Working|    Higher education|      Civil marriage|

In [ ]:
df_revised.createOrReplaceTempView('temporary_view')

In [ ]:
spark.sql('''
SELECT *
FROM temporary_view
WHERE CNT_CHILDREN = 4
''').show()

+-----+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+-------------+-----------------+---+--------------+---------------+---------------+
|  _c0|     ID|GENDER|FLAG_OWN_CAR|FLAG_OWN_PROPERTY|CNT_CHILDREN|AMT_INCOME_TOTAL|         INCOME_TYPE|      EDUCATION_TYPE|FAMILY_STATUS|     HOUSING_TYPE|AGE|YEARS_EMPLOYED|CNT_FAM_MEMBERS|APPROVAL_STATUS|
+-----+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+-------------+-----------------+---+--------------+---------------+---------------+
| 1405|5113188|     M|           1|                1|           4|        292500.0|             Working|Secondary / secon...|      Married|House / apartment| 39|             1|              6|              1|
| 3495|5028252|     M|           1|                0|           4|        202500.0|Commercial associate|Secondary / secon...|      Married|House / apartment| 42|   

In [ ]:
spark.sql('''
SELECT *
FROM temporary_view
WHERE CNT_CHILDREN = 4
''').show()

+-----+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+-------------+-----------------+---+--------------+---------------+---------------+
|  _c0|     ID|GENDER|FLAG_OWN_CAR|FLAG_OWN_PROPERTY|CNT_CHILDREN|AMT_INCOME_TOTAL|         INCOME_TYPE|      EDUCATION_TYPE|FAMILY_STATUS|     HOUSING_TYPE|AGE|YEARS_EMPLOYED|CNT_FAM_MEMBERS|APPROVAL_STATUS|
+-----+-------+------+------------+-----------------+------------+----------------+--------------------+--------------------+-------------+-----------------+---+--------------+---------------+---------------+
| 1405|5113188|     M|           1|                1|           4|        292500.0|             Working|Secondary / secon...|      Married|House / apartment| 39|             1|              6|              1|
| 3495|5028252|     M|           1|                0|           4|        202500.0|Commercial associate|Secondary / secon...|      Married|House / apartment| 42|   

In [ ]:
spark.sql('''
SELECT GENDER, AVG(YEARS_EMPLOYED)
FROM temporary_view
WHERE INCOME_TYPE = 'Commercial associate'
GROUP By GENDER
''').show()

+------+-------------------+
|GENDER|avg(YEARS_EMPLOYED)|
+------+-------------------+
|     F|  6.630183727034121|
|     M|  5.719960765080922|
+------+-------------------+



# Forest Model

0.9857305010539971


In [ ]:
pd.DataFrame({
    'feature': X.columns,
    'importance': forest.feature_importances_
}).sort_values(by='importance', ascending=False)

,feature,importance
4,AGE,0.254868
3,AMT_INCOME_TOTAL,0.251144
5,YEARS_EMPLOYED,0.155613
1,FLAG_OWN_PROPERTY,0.042076
6,CNT_FAM_MEMBERS,0.038257
0,FLAG_OWN_CAR,0.035297
2,CNT_CHILDREN,0.028495
18,EDUCATION_TYPE_Secondary / secondary special,0.018568
13,INCOME_TYPE_Working,0.018032
9,INCOME_TYPE_Commercial associate,0.017131


In [ ]:
AMT_INCOME_TOTAL
YEARS_EMPLOYED
FLAG_OWN_PROPERTY
CNT_FAM_MEMBERS
FLAG_OWN_CAR
CNT_CHILDREN